In [ ]:
import pandas as pd
import datetime as dt
import psycopg2 as pg
import pandas.io.sql as psql
import warnings
warnings.filterwarnings('ignore')

In [ ]:
DbInternal = pg.connect(DbInternal)
DbMain = pg.connect(DbMain)

start_date = "2024-02-04" #exclusive

In [ ]:
traffic = psql.read_sql('''SELECT * FROM db_gsc_search_data WHERE grass_date > '%s'
''' % (start_date), DbInternal)

main_web_from_blog = psql.read_sql('''SELECT to_char(grass_date + interval '8 hours', 'DD/MM/YYYY') as grass_date, main_web_from_blog as clicks FROM db_gsc_search_overall_data WHERE date(grass_date + interval '8 hours') > date('%s')
''' % (start_date), DbInternal)

In [ ]:
main_web_from_blog = psql.read_sql('''SELECT to_char(grass_date + interval '8 hours', 'DD/MM/YYYY') as grass_date, main_web_from_blog as clicks FROM db_gsc_search_overall_data WHERE date(grass_date + interval '8 hours') > date('%s')
''' % (start_date), DbInternal)

In [ ]:
main_web_from_blog['grass_date'] = pd.to_datetime(main_web_from_blog['grass_date'],errors='coerce', format ='%d/%m/%Y')
main_web_from_blog = main_web_from_blog.dropna().sort_values(by='grass_date')
main_web_from_blog = main_web_from_blog[main_web_from_blog['grass_date'] > start_date]

main_web_from_blog['category'] = "Main Traffic from Blog"
main_web_from_blog['week'] = main_web_from_blog['grass_date'].dt.to_period('W')
main_web_from_blog_group = main_web_from_blog.groupby(['category', 'week'])['clicks'].sum().reset_index()
main_web_from_blog_group_pivot = main_web_from_blog_group.pivot_table(values = 'clicks', index = 'category', columns = 'week')

In [ ]:
def categorize(row):
    landing_page = row['landing_page']
    product_name = row['product_name']

    if "novelship.com" in landing_page or "login" in landing_page or "mobile-app" in landing_page or "signup" in landing_page:
        return "Homepage"
    elif "careers" in landing_page:
        return "Careers"
    elif "lookbook" in landing_page:
        return "Lookbook"
    elif "news/" in landing_page or "news-tw/" in landing_page:
        return "SEO article"
    elif any(keyword in landing_page for keyword in ["privacy", "info", "instruction", "support", "authentication", "about", "article", "authenticateforme"]):
        return "Support"
    elif landing_page == "sell" or landing_page == "terms" or landing_page == "tokutei" or landing_page == "tw-branches":
        return "Support"
    elif "?country=" in landing_page and "?size=" in landing_page:
        return "GMC"
    elif "?country=" in landing_page:
        return "Product"
    elif any(keyword in landing_page for keyword in ["sneakers", "apparel", "collectibles", "trading-cards", "size-wishlist"]):
        return "Product"
    else:
        return "Product"

traffic['category'] = traffic.apply(categorize, axis=1)

In [ ]:
traffic['grass_date'] = pd.to_datetime(traffic['grass_date'],errors='coerce', infer_datetime_format=True)

In [ ]:
traffic['week'] = traffic['grass_date'].dt.to_period('W')
traffic_group = traffic.groupby(['category','week'])['clicks'].sum().reset_index()
traffic_group = traffic_group.groupby(['week']).apply(lambda x: x.sort_values(by='clicks', ascending=False)).reset_index(drop=True)
pivot_traffic = traffic_group.pivot_table(values='clicks', index='category', columns='week')
pivot_traffic = pivot_traffic.fillna(0)

In [ ]:
import zipfile
import pandas as pd

# https://search.google.com/u/0/search-console/performance/search-analytics?resource_id=???
zip_file_path = "Directory.zip"
csv_file_name = "Dates.csv" #in zip folder

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extract(csv_file_name, "C:/Directory")

csv_file_path = "C:/Directory" + csv_file_name
discover = pd.read_csv(csv_file_path)

In [ ]:
discover['Date'] = pd.to_datetime(discover['Date'], errors = 'coerce', infer_datetime_format=True)
discover = discover[discover['Date'] > start_date]
discover['week'] = discover['Date'].dt.to_period('W')
discover['category'] = "Discover"
discover_group = discover.groupby(['category','week'])['Clicks'].sum().reset_index()
discover_group = discover_group.groupby(['week']).apply(lambda x: x.sort_values(by='Clicks', ascending=False)).reset_index(drop=True)
discover_traffic = discover_group.pivot_table(values='Clicks', index='category', columns='week')

In [ ]:
#https://merchants.google.com/mc/reporting/editor?a=144698196&reportId=6181392765&authuser=0
gmc_free_listings = pd.read_csv("C:/GMC_Clicks.csv")
gmc_paid = gmc_free_listings[gmc_free_listings['Program'] != "Free listings"]
gmc_free_listings = gmc_free_listings[gmc_free_listings['Program'] == "Free listings"]


In [ ]:
gmc_free_listings['Date'] = pd.to_datetime(gmc_free_listings['Day'], errors = 'coerce', infer_datetime_format=True)
gmc_free_listings = gmc_free_listings[gmc_free_listings['Date'] > start_date]
gmc_free_listings['category'] = "GMC"
gmc_free_listings['week'] = gmc_free_listings['Date'].dt.to_period('W')
gmc_group = gmc_free_listings.groupby(['category', 'week'])['Clicks'].sum().reset_index()
gmc_group_pivot = gmc_group.pivot_table(values = 'Clicks', index = 'category', columns = 'week')

In [ ]:
gmc_paid['Date'] = pd.to_datetime(gmc_paid['Day'], errors = 'coerce', infer_datetime_format=True)
gmc_paid = gmc_paid[gmc_paid['Date'] > start_date]
gmc_paid['category'] = "GMC Paid Traffic"
gmc_paid['week'] = gmc_paid['Date'].dt.to_period('W')
gmc_paid = gmc_paid.groupby(['category', 'week'])['Clicks'].sum().reset_index()
gmc_paid_pivot = gmc_paid.pivot_table(values = 'Clicks', index = 'category', columns = 'week')

In [ ]:
export_df = pd.concat([pivot_traffic, discover_traffic, gmc_group_pivot, main_web_from_blog_group_pivot], axis = 0)
custom_order = [ "GMC", "Product", "Homepage", "Support", "Careers", "Lookbook", "SEO article", "Discover", "Main Traffic from Blog"]
export_df = export_df.reindex(custom_order)

In [ ]:
total_skus = psql.read_sql('''SELECT
to_char(p.created_at + interval '8 hours', 'DD/MM/YYYY') as creation_date,
count(p.id) as products
FROM product p
GROUP BY 1 
''', DbMain)

DbMain.close()
DbInternal.close()

In [ ]:
total_skus['creation_date'] = pd.to_datetime(total_skus['creation_date'], format = '%d/%m/%Y')
#total_skus_bef = total_skus[total_skus['creation_date'] <= start_date]
total_skus_after = total_skus[total_skus['creation_date'] > start_date]
#total_skus_bef['week'] = "bef 2024-02-05"
#total_skus_bef = total_skus_bef.groupby('week')['products'].sum().reset_index()


total_skus_after['week'] = total_skus_after['creation_date'].dt.to_period('W')
total_skus_after = total_skus_after.groupby('week')['products'].sum().sort_index(ascending = False).reset_index()

In [ ]:
#total_skus_grouped = pd.concat([total_skus_after, total_skus_bef], axis = 0)
total_skus_after['category'] = 'Total SPU'
total_skus_after = total_skus_after.pivot_table(values = 'products', index = 'category', columns = 'week')

In [ ]:
total_skus[total_skus['creation_date'] <= start_date]['products'].sum()

In [ ]:
import gspread
import gspread_dataframe as gd

gc1 = gspread.service_account('service_account.json')
sh = gc1.open_by_url('https://docs.google.com/spreadsheets/d/1eP7Tb85W0w8vRssRn5z9L7l_gAAYBgmaUi0yT2F3cLY/edit?pli=1#gid=1643402199')
sh1 = sh.worksheet("0. SEO (ref)")

In [ ]:
export_df = export_df.fillna(0)
# For main Organic
export_values = [[index] + row.tolist() for index, row in export_df.iterrows()] #to include row name
sh1.update('AB24', export_values)

In [ ]:
# For paid GMC
export_values = [[index] + row.tolist() for index, row in gmc_paid_pivot.iterrows()] 
sh1.update('AB9', export_values)

In [ ]:
# For Total SKUs listings on Novelship
export_values = [[index] + row[1:].tolist() for index, row in total_skus_after.iterrows()] 
sh1.update('AB41', export_values)